In [ ]:
sc

In [ ]:
import os

# Please update your alias belpw (or any custom directory you uploaded the samples to).
# You can find the name from the directory structure in the left nav
root_dir = "./Users/<your_user_alias>/featurestore_sample"

if os.path.isdir(root_dir):
    print("The folder exists.")
else:
    print("The folder does not exist. Please create or fix the path")

In [ ]:
from azureml.featurestore import create_feature_set_spec

dsl_featureset_spec_path = (
    root_dir + "/featurestore/featuresets/taxi"
)

fss = create_feature_set_spec(spec_path=dsl_featureset_spec_path)


In [ ]:
print(fss)

In [ ]:
spark_df = fss.to_spark_dataframe_dsl(feature_window_end_date_time="2021-01-31")

In [ ]:
spark_df.toString()

In [ ]:
spark_df.getDataframe().show()

In [ ]:
print(fss._to_feathr_config("2021-01-31")[0])
print(fss._to_feathr_config("2021-01-31")[0])